In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
from custom import weather

matplotlib.rcParams['font.family'] = ['PingFang HK']
def rmax(maxrow: int=50):
    pd.set_option('display.max_rows', maxrow)
def cmax(maxcol: int=50):
    pd.set_option('display.max_columns', maxcol)

%load_ext autoreload
%autoreload 2

In [2]:
#load station information
df = pd.read_csv('all_station_20201126.csv')
df.shape

(608, 16)

In [3]:
#load weather records
df_record = pd.read_pickle('weather_allstation_20150101-20201227.bz2')

In [4]:
## get a subset of data from Dec 2020, and only the precipitation column
df_sub = df_record.loc[pd.IndexSlice[:, '2020-12':], 'Precp']

In [5]:
df_w = df_sub.apply(pd.to_numeric, errors='coerce')
df_w.index.names = ['stn_code', 'date']
## remove the trailing empty records
df_w.dropna(how='all', inplace=True)

In [6]:
missing = df_w.isna().groupby('stn_code').sum()
missing_codes = missing[missing>0].index
df[df.stn_code.isin(missing_codes)]

,stn_code,stn_name,altitude,Longitude,Latitude,city,address,data_start_date,stn_end_date,comment,orig_stn_code,new_stn_code,data_start_date2,data_duration,data_duration2,data_period


In [7]:
df_w.index.get_level_values(0).unique()

Index(['466880', '466900', '466910', '466920', '466930', '466940', '466950',
       '466990', '467050', '467060',
       ...
       'C1V600', 'C1V780', 'C1X040', 'C1Z030', 'C1Z040', 'C1Z110', 'C1Z120',
       'C1Z130', 'C1Z140', 'C1Z240'],
      dtype='object', name='stn_code', length=603)

In [8]:
df_w

stn_code  date      
466880    2020-12-01    16.5
          2020-12-02     6.0
          2020-12-03     7.0
          2020-12-04     2.5
          2020-12-08     3.5
                        ... 
C1Z240    2020-12-23    24.5
          2020-12-24    23.0
          2020-12-25     9.5
          2020-12-26     0.0
          2020-12-27     6.5
Name: Precp, Length: 16160, dtype: float64

In [9]:
df_combined = pd.merge(df.iloc[:, :5], df_w.reset_index(), how='inner', on='stn_code').set_index('date')
df_combined

,stn_code,stn_name,altitude,Longitude,Latitude,Precp
date,,,,,,
2020-12-01,466880,板橋,9.7,121.442017,24.997647,16.5
2020-12-02,466880,板橋,9.7,121.442017,24.997647,6.0
2020-12-03,466880,板橋,9.7,121.442017,24.997647,7.0
2020-12-04,466880,板橋,9.7,121.442017,24.997647,2.5
2020-12-08,466880,板橋,9.7,121.442017,24.997647,3.5
...,...,...,...,...,...,...
2020-12-23,C1Z240,中平林道,1163.0,121.267630,23.421150,24.5
2020-12-24,C1Z240,中平林道,1163.0,121.267630,23.421150,23.0
2020-12-25,C1Z240,中平林道,1163.0,121.267630,23.421150,9.5


In [138]:
temp = df_combined.loc['2020-12-01']

In [139]:
import folium
import branca
import colorcet as cc
from folium import plugins
from scipy.interpolate import griddata
import geojsoncontour
import scipy as sp
import scipy.ndimage

# Setup
precp_mean = temp.Precp.mean()
precp_std  = temp.Precp.std()
precp_min = temp.Precp.min()
precp_max = temp.Precp.max()
debug     = False

In [140]:
# Create the colormap
nc = 12
# cm = matplotlib.cm.jet
cm = cc.cm.rainbow
upper = cm(np.round(np.linspace(0, 256, 3*nc//4)).astype('int'))
lower = np.ones((int(nc//4)+2,4))
lower[0,3] = 0.
for i in range(3):
    lower[:,i] = np.linspace(1, upper[0,i], lower.shape[0])

cmap_raw = np.vstack(( lower[1:-1,:], upper ))
levels = cmap_raw.shape[0]
cmap = matplotlib.colors.ListedColormap(cmap_raw, name='myColorMap', N=levels)
# chex = [matplotlib.colors.rgb2hex(cmap(i)) for i in range(nc)]
cmap2 = [cmap(i) for i in range(levels)]

vmin = temp.Precp.min()
vmax = temp.Precp.max()
colorbar = branca.colormap.LinearColormap(cmap2, vmin=3, vmax=vmax).to_step(levels)
colorbar

In [141]:
# The original data
x_orig = np.asarray(temp.Longitude.tolist())
y_orig = np.asarray(temp.Latitude.tolist())
z_orig = np.asarray(temp.Precp.tolist())

n=500
# Make a grid
x_arr          = np.linspace(np.min(x_orig), np.max(x_orig), n)
y_arr          = np.linspace(np.min(y_orig), np.max(y_orig), n)
x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)

# Grid the values
z_mesh = griddata((x_orig, y_orig), z_orig, (x_mesh, y_mesh), method='linear', fill_value=np.nan)

In [142]:
from scipy.interpolate.interpnd import _ndim_coords_from_arrays
from scipy.spatial import cKDTree

# Construct kd-tree, functionality copied from scipy.interpolate
xy = np.array(list(zip(x_orig, y_orig)))
tree = cKDTree(xy)
xi = _ndim_coords_from_arrays((x_mesh, y_mesh), ndim=xy.shape[1])
dists, indexes = tree.query(xi, k=1)

In [143]:
THRESHOLD = 0.1
z_mesh2 = z_mesh.copy()
z_mesh2[dists > THRESHOLD] = np.nan

z_mesh2[z_mesh2<3] = np.nan
### Gaussian filter the grid to make it smoother
# sigma = [0.5, 0.5]
# z_mesh3 = sp.ndimage.filters.gaussian_filter(z_mesh2, sigma, mode='constant')


In [144]:
# Create the contour
%matplotlib widget

contourf = plt.contourf(x_mesh, y_mesh, z_mesh2, levels, alpha=1, colors=cmap2, linestyles='None', vmin=1, vmax=vmax)
plt.plot(temp.Longitude, temp.Latitude, 'ro', alpha=0.2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [145]:
# Convert matplotlib contourf to geojson
geojson = geojsoncontour.contourf_to_geojson(
    contourf=contourf,
    min_angle_deg=None,
    ndigits=5,
    stroke_width=0,
    fill_opacity=1)

# Set up the folium plot
geomap = folium.Map([temp.Latitude.mean(), temp.Longitude.mean()], zoom_start=8, tiles="openstreetmap")

# Plot the contour plot on folium
folium.GeoJson(
    geojson,
    style_function=lambda x: {
        'color':     x['properties']['stroke'],
        'weight':    x['properties']['stroke-width'],
#         'weight': 0,
        'fillColor': x['properties']['fill'],
        'fillOpacity': 0. if x['properties']['fill']=='#ffffff' else .7,
    }).add_to(geomap)
# add the stations on map
for i, row in temp.iterrows():
    hovertext = f'{row.stn_name}, Alt {row.altitude}m, {row.Precp}'
    folium.CircleMarker(
            [row.Latitude, row.Longitude],
            radius=3,
            popup=None,
            tooltip=hovertext,
            color='#FF0000',
            weight=0,
            fill=True,
            fill_color='#FF0000',
            fill_opacity=0.5,
            parse_html=False).add_to(geomap)

# Add the colormap to the folium map
colorbar.caption = 'Precipitation'
geomap.add_child(colorbar)

# Fullscreen mode
# plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)
geomap

In [127]:
import numpy as np
import pandas as pd
import folium
import branca
from folium import plugins
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import geojsoncontour
import scipy as sp
import scipy.ndimage

# Setup
temp_mean = 12
temp_std  = 2
debug     = False

# Setup colormap
colors = ['#d7191c',  '#fdae61',  '#ffffbf',  '#abdda4',  '#2b83ba']
vmin   = temp_mean - 2 * temp_std
vmax   = temp_mean + 2 * temp_std
levels = len(colors)
cm     = branca.colormap.LinearColormap(colors, vmin=vmin, vmax=vmax).to_step(levels)

# Create a dataframe with fake data
df = pd.DataFrame({
    'longitude':   np.random.normal(11.84,     0.15,     1000),
    'latitude':    np.random.normal(55.55,     0.15,     1000),
    'temperature': np.random.normal(temp_mean, temp_std, 1000)})

# The original data
x_orig = np.asarray(df.longitude.tolist())
y_orig = np.asarray(df.latitude.tolist())
z_orig = np.asarray(df.temperature.tolist())

# Make a grid
x_arr          = np.linspace(np.min(x_orig), np.max(x_orig), 500)
y_arr          = np.linspace(np.min(y_orig), np.max(y_orig), 500)
x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)

# Grid the values
z_mesh = griddata((x_orig, y_orig), z_orig, (x_mesh, y_mesh), method='linear')

# Gaussian filter the grid to make it smoother
sigma = [5, 5]
z_mesh = sp.ndimage.filters.gaussian_filter(z_mesh, sigma, mode='constant')

# Create the contour
contourf = plt.contourf(x_mesh, y_mesh, z_mesh, levels, alpha=0.5, colors=colors, linestyles='None', vmin=vmin, vmax=vmax)

# Convert matplotlib contourf to geojson
geojson = geojsoncontour.contourf_to_geojson(
    contourf=contourf,
    min_angle_deg=3.0,
    ndigits=5,
    stroke_width=1,
    fill_opacity=0.5)

# Set up the folium plot
geomap = folium.Map([df.latitude.mean(), df.longitude.mean()], zoom_start=10, tiles="cartodbpositron")

# Plot the contour plot on folium
folium.GeoJson(
    geojson,
    style_function=lambda x: {
        'color':     x['properties']['stroke'],
        'weight':    x['properties']['stroke-width'],
        'fillColor': x['properties']['fill'],
        'opacity':   0.6,
    }).add_to(geomap)

# Add the colormap to the folium map
cm.caption = 'Temperature'
geomap.add_child(cm)

# Fullscreen mode
plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)

In [89]:
geomap

In [45]:
# Plot the data
geomap.save(f'data/folium_contour_temperature_map.html')

In [87]:
!pip install geojsoncontour

     |████████████████████████████████| 736 kB 1.2 MB/s eta 0:00:01
